In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cd /content/drive/MyDrive/Projects/GO_GO_Nihongo

/content/drive/.shortcut-targets-by-id/14kN-FziW24aUnE1LuiZ3m2W5UaK7G2vt/GO_GO_Nihongo


In [1]:
%pip install mojimoji
%pip install sentencepiece
%pip install nltk
%pip install scikit-learn
%pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout, Input, dot, Activation, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import optimizers
from tensorflow.keras import initializers, regularizers, constraints

from sklearn.model_selection import train_test_split


import mojimoji
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import io

import nltk
import unicodedata
import sentencepiece as spm

# ignore warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(tf.__version__)
# gpu
tf.test.is_gpu_available()

2.18.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


False

# load text file

this dataset is aleady implemented a SentenceSpace

In [3]:
num_example = 300000

# create each languages list
def create_lang_list(num_example):
    # load txt file
    lines =  io.open("datas/raw.txt", encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_example]]

    return zip(*word_pairs)

In [4]:
en, ja = create_lang_list(num_example)

**SentencePiece**

using pretrain model


In [5]:
ja_sentence = list()
for i in ja:
    ja_sentence.append(i.replace(" ", ""))

In [28]:
ja_text = list()
# sp = spm.SentencePieceProcessor()
sp =spm.SentencePieceProcessor(model_file='./Pretrained tokenizer/spm.en.nopretok.model')
# sp.Load("wiki-ja.model")
for text in ja_sentence:
    ja_text.append(" ".join(sp.EncodeAsPieces(text)).replace("▁", "").strip())

In [29]:
ja_text

['あ な た は 戻 っ た の ね ハ ロ ル ド ?',
 '俺 の 相 手 は シ ャ ー ク だ 。',
 '引 き 換 え だ あ る 事 と あ る 物 の',
 'も う い い よ ご ち そ う さ ま う う ん',
 'も う 会 社 に は 来 な い で く れ 電 話 も する な',
 'き れ い だ 。',
 '連 れ て 行 け 殺 し そ う だ わ か っ た か ?',
 '殺 し た の か !',
 'わ ぁ ~ ! い つ も す み ま せ ん 。 い い の よ ~ 。',
 'カ ン パ ニ ー の 元 社 員 が',
 'じ ゃ ア イ ツ ラ は ど こ ?',
 '相 手 を 陥 れ る と か ...',
 '必 要 の な い 子 耐 え る に して い る 。',
 '犯 人 像 と は 違 和 感 の あ る 見 た 目 だ が',
 'あ !',
 'も っ と 詳 し い の 見 せ ます から 。',
 'そ れ 今 持 ち 出 す か ?',
 'じ ゃ あ 仕 事 終 わ っ た ら 医 局 に 来 て よ 。',
 'あ の ... 皆 さ ん で 先 済 ま せ て く だ さ い 。',
 '六 花 ち ゃ ん は ず っ と ず っ と ず っ と',
 'ほ ら 別 に 変 な こ と じ ゃ な い で し ょ ?',
 '変 な 子 供 服',
 '船 長 から 他 の 誰 か が 扱 っ た 形 跡 が な い か d na を 再 調 査 する よ う に 頼 ま れ ま し た',
 'も し 残 り も 上 手 く 書 け た ら',
 '以 前 に 増 して 魅 力 的 だ',
 '也 就 是 说 ... 该 不 会 是 ...',
 '長 い こ と 友 達 な の に 気 付 か な か っ た の か ?',
 'ビ ル の ID 信 号 も 5 分 前 に 消 え た わ',
 '口 の 中 に 血 が 見 え る',
 '" AS A MOTO は こ の ま ま じ ゃ い け な い と 思 う 。 "',
 '嵐 に は 嵐 の',
 'な る ほ ど 簡 単 に ゃ 入 れ そ う に ね え な',
 'ニュー ス は ? 確 か に 二 人

# Text Pre-processing

 **Removing accented characters**
 e.g. é → e.


**remove special word**
e.g. remove "123#@!"


**normalize japanese**
e.g.  カナダ  → ｶﾅﾀﾞ

**Tokenize**
 e.g. I am going to restaurant → [[I], [am], [going], [to], [restaurant]]


# normalize Japanese (Hiragana and Katakana)

In [30]:
def moji(text):
  return mojimoji.zen_to_han(text)

In [31]:
print("カナダ →", moji("カナダ"))

カナダ → ｶﾅﾀﾞ


# Removing accented characters

English might have accent like é but Japanese doesn't have any accent
I just create different function to ascii for Japanese and English

In [32]:
# Removing accented characters
def english_unicode_to_ascii(text):
     return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore'))

def japanese_unicode_to_ascii(text):
    return ''.join(ascii_text for ascii_text in unicodedata.normalize('NFKD', text))

In [33]:
# e.g.
japanese_unicode_to_ascii("こんにちは。 今日は"), english_unicode_to_ascii("Hello world é ")

('こんにちは。 今日は', 'Hello world e ')

# remove special characters and create space between word and punctuation

replacing everything with space except(a-z, A-Z, ".",  "?",  "!",  ",", "-", "ー", , "。", "Kanji", "Katakana", "Hiragana")
create space between word and punctuation (? ! . , 、 。)

In [34]:
def replace_special_character_to_space_en(text):
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)

    text = re.sub(r"[^a-zA-Z?.!,¿]+", " ", text)

    text = text.strip()

    return text

def replace_special_character_to_space(text):
    text = re.sub(r"([?!。、¿])", r" \1", text)
    pattern = r"[^\u3041-\u309F\u30A1-\u30FF\uFF66-\uFF9F\u4E00-\u9FD0\u309B\u3099\uFF9E\u309C\u309A\uFF9F?!\s、。.,0-9]+"
    text = re.sub(pattern, '', text).rstrip().strip()
    text = re.sub(r'[" "]+', " ", text)
    text = text.replace("・" , "")

    text = text.lower()

    return text

In [35]:
# e.g.
replace_special_character_to_space("hello #@…123world."), replace_special_character_to_space("こん・にちは・いい天気。")

('123.', 'こんにちはいい天気 。')

# Text normalize

I will normalize English text and Japanese text using function which is defined so far

In [36]:
def normalize_english(english_text, japanese_text):

    input_value = []
    target_value = []

    for en_text, ja_text in zip(english_text, japanese_text):

        # normalize English
        en_text = english_unicode_to_ascii(en_text)
        en_text = replace_special_character_to_space_en(en_text)

        en_text = "start_ " + en_text + " _end"
        # input value doesn't need  a START and END sentence
        input_value.append(en_text)

        # normalize Japanese
        ja_text = japanese_unicode_to_ascii(ja_text)
        ja_text = replace_special_character_to_space(ja_text)
        ja_text = moji(ja_text)

        # add StTART and END sentence
        ja_text = "start_ " + ja_text + " _end"

        target_value.append(ja_text)

    return input_value, target_value

In [37]:
# get normalize text data
target_value, input_value = normalize_english(en, ja_text)

# convert to Series
x = pd.Series(input_value)
y = pd.Series(target_value)

In [38]:
pd.DataFrame({"input": x, "target": y}).head(10)

,input,target
0,start_ あ な た は 戻 っ た の ね ﾊ ﾛ ﾙ ﾄ゙ ? _end,"start_ you are back , aren t you , harold ? _end"
1,start_ 俺 の 相 手 は ｼ ｬ ｰ ｸ だ ｡ _end,start_ my opponent is shark . _end
2,start_ 引 き 換 え だ あ る 事 と あ る 物 の _end,start_ this is one thing in exchange for anoth...
3,start_ も う い い よ ご ち そ う さ ま う う ん _end,"start_ yeah , i m fine . _end"
4,start_ も う 会 社 に は 来 な い で く れ 電 話 も する な _end,start_ don t come to the office anymore . don ...
5,start_ き れ い だ ｡ _end,start_ looks beautiful . _end
6,start_ 連 れ て 行 け 殺 し そ う だ わ か っ た か ? _end,"start_ get him out of here , because i will fu..."
7,start_ 殺 し た の か ! _end,start_ you killed him ! _end
8,start_ わ ぁ ! い つ も す み ま せ ん ｡ い い の よ ｡ _end,"start_ okay , then who ? _end"
9,start_ ｶ ﾝ ﾊ゚ ﾆ ｰ の 元 社 員 が _end,start_ it seems a former employee . . . _end


# tokenize
tokenize each language word based on space

In [39]:
def tokenize(lang):
    # vectorize a text corpus
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters=' ')

    # updates internal vocabulary based on a list of texts
    # e.g. "[this place is good ]"→{this:2, place:3, is:1, good:4} "
    lang_tokenizer.fit_on_texts(lang)

    # Transforms each text in texts to a sequence of integers.
    # e.g. this place is good → [[2, 3, 1, 4]]
    tensor = lang_tokenizer.texts_to_sequences(lang)

    # transform a list of num sample into a 2D Numpy array of shape
    # Fixed length because length of sequence of integers are different
    # return (len(sequences), maxlen)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    return tensor, lang_tokenizer

In [40]:
# e.g.
tokenize(['this place is good', "こんにちは 今日は いい天気 。", "today is so cold"])

(array([[ 2,  3,  1,  4],
        [ 5,  6,  7,  8],
        [ 9,  1, 10, 11]]),
 <keras.src.legacy.preprocessing.text.Tokenizer at 0x21c1b865750>)

# create clean dataset

In [41]:
# cleate a clean dataset
def create_dataset(en, ja):

    # input_tensor, target_tensor: 2d numpy array
    # input_lang_tokenize, target_lang_tokenize: word dictionary
    input_tensor, input_lang_tokenize = tokenize(en)
    target_tensor, target_lang_tokenize = tokenize(ja)

    return input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize

In [42]:
# Ensure x and y are properly defined and contain data
if not x.empty and not y.empty:
	input_tensor, target_tensor, input_lang_tokenize, target_lang_tokenize = create_dataset(x, y)
else:
	print("Error: x and y are empty. Please check the data.")

In [43]:
def max_length(input_tensor, target_tensor):

    # max length of input sentense and target sentense
    english_len = [len(i) for i in input_tensor]

    japanese_len = [len(i) for i in target_tensor]

     # print max length
    print("english length:", max(english_len))
    print("japanese length:", max(japanese_len))
    max_len_input =  max(english_len)
    max_len_target =  max(japanese_len)

    return max_len_input, max_len_target

In [44]:
# Calculate max_length of the target tensors
max_length_input, max_length_target = max_length(input_tensor, target_tensor)

english length: 145
japanese length: 69


In [45]:
# create trainnig set and validation set
X_train, X_test, \
    Y_train, Y_test = train_test_split(input_tensor, target_tensor, test_size=0.2, shuffle=True)

X_test, X_val, \
    Y_test, Y_val = train_test_split(X_test, Y_test, test_size=0.5, shuffle=True)


# show length
print(len(X_train), len(Y_train), len(X_test), len(Y_test), len(X_val), len(Y_val))

240000 240000 30000 30000 30000 30000


In [46]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            # Index number assigned to each word
            print("%d----->%s" % (t, lang.index_word[t]))

In [47]:
print("input lang: index to word mapping")
convert(input_lang_tokenize, X_train[10])
print("output lang: index to word mapping")
convert(target_lang_tokenize, Y_train[10])

input lang: index to word mapping
1----->start_
314----->止
61----->め
17----->る
18----->か
24----->?
2----->_end
output lang: index to word mapping
1----->start_
9----->to
641----->win
16----->!
2----->_end


# Encoder Model

In [48]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size, dropout_rate):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.dropout = Dropout(dropout_rate)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.first_lstm = tf.keras.layers.LSTM(self.enc_units,
                                                            return_sequences=True,
                                                            recurrent_initializer='glorot_uniform')

        self.final_lstm = tf.keras.layers.LSTM(self.enc_units,
                                                    return_sequences=True,
                                                    return_state=True,
                                                    recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.first_lstm(x, initial_state =hidden)
        output, state_h, state_c = self.final_lstm(x)
        state = [state_h, state_c ]

        return output, state

    def initialize_hidden_state(self):
            return tf.zeros((self.batch_size , self.enc_units)), tf.zeros((self.batch_size , self.enc_units))

# attention

In [49]:
class Attention(tf.keras.models.Model):

    def __init__(self, units: int, *args, **kwargs):

        super().__init__(*args, **kwargs)
        self.units = units

        self.q_dense_layer = Dense(units, use_bias=False, name='q_dense_layer')
        self.k_dense_layer = Dense(units, use_bias=False, name='k_dense_layer')
        self.v_dense_layer = Dense(units, use_bias=False, name='v_dense_layer')
        self.output_dense_layer = Dense(units, use_bias=False, name='output_dense_layer')

    def call(self, input, memory):

        q = self.q_dense_layer(input)
        k = self.k_dense_layer(memory)
        v = self.v_dense_layer(memory)

        depth = self.units // 2
        q *= depth ** -0.5  # for scaled dot production

        # caluclate relation between query and key
        logit = tf.matmul(q, k, transpose_b=True)

        attention_weight = tf.nn.softmax(logit)

        attention_output = tf.matmul(attention_weight, v)
        return self.output_dense_layer(attention_output)

# Decoder Model

In [50]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_size, dropout_rate):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.dropout = Dropout(dropout_rate)
        self.first_lstm = tf.keras.layers.LSTM(self.dec_units,
                                                            return_sequences=True)
        self.final_lstm = tf.keras.layers.LSTM(self.dec_units,
                                                            return_sequences=True,
                                                            return_state=True)

        self.fc = tf.keras.layers.Dense(vocab_size)

        self.attention = Attention(self.dec_units)

    def call(self, x, hidden, enc_output):
        x = self.embedding(x)
        x = self.dropout(x)

        x =  self.first_lstm(x)
        output, state_h, state_c = self.final_lstm(x)
        state = [state_h, state_c]
        attention_weights = self.attention(output, enc_output)
        output = tf.concat([output, attention_weights], axis=-1)


        output = tf.reshape(output, (-1, output.shape[2]))

        output = self.fc(output)

        return  output, state

# optimizer and the loss function

In [51]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.9, epsilon=1e-04, decay=1e-06)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

e:\7th sem\GO_GO_Nihongo\venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


# define parameter

In [52]:
# BUFFER_SIZE >= dataset if smaller than dataset can't shuffle equally
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 256
# BATCH_SIZE = 32
dropout_rate = 0.3
# if None steps_per_epoch == mum of dataset
train_steps_per_epoch = len(X_train) // BATCH_SIZE
val_steps_per_epoch = len(X_val) // BATCH_SIZE
print("train step %d" % train_steps_per_epoch)
embedding_dim = 300
units = 128
# units = 512
vocab_inp_size = len(input_lang_tokenize.word_index) + 1
print('Total unique words in the input: %s' % len(input_lang_tokenize.word_index))
print('Total unique words in the target: %s' % len(target_lang_tokenize.word_index))
vocab_tar_size = len(target_lang_tokenize.word_index) + 1

# create train dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# create validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(BATCH_SIZE, drop_remainder=True)

train step 937
Total unique words in the input: 4659
Total unique words in the target: 52080


In [53]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE, dropout_rate)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE, dropout_rate)

# Checkpoints

In [54]:
checkpoint_dir = './train_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

# train model

In [55]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([target_lang_tokenize.word_index['start_']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the sladecoder
      predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [56]:
# # trained model for 45 epochs
# EPOCHS = 5

# for epoch in range(EPOCHS):


#   enc_hidden = encoder.initialize_hidden_state()
#   train_loss = 0
#   val_loss = 0

#   for (batch, (inp, targ)) in enumerate(train_dataset.take(train_steps_per_epoch)):
#     train_batch_loss = train_step(inp, targ, enc_hidden)
#     train_loss += train_batch_loss


#   for (batch, (val_inp, val_tar)) in enumerate(val_dataset.take(val_steps_per_epoch)):
#     val_batch_loss = train_step(val_inp, val_tar, enc_hidden)
#     val_loss += val_batch_loss


#   # saving (checkpoint) the model every 2 epochs
#   if (epoch + 1) % 2 == 0:
#     checkpoint.save(file_prefix = checkpoint_prefix)

#   print('Epoch {} Train Loss {:.4f}'.format(epoch + 1,
#                                       train_loss / train_steps_per_epoch))
#   print('Epoch {} Validation Loss {:.4f}'.format(epoch + 1,
#                                       val_loss / val_steps_per_epoch))
from tqdm import tqdm

# Restore the latest checkpoint if it exists
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
if latest_checkpoint:
    checkpoint.restore(latest_checkpoint)
    print(f"Restored from {latest_checkpoint}")
else:
    print("Starting training from scratch.")

EPOCHS = 15

for epoch in range(EPOCHS):

    enc_hidden = encoder.initialize_hidden_state()
    train_loss = 0
    val_loss = 0

    print(f"\nEpoch {epoch + 1}/{EPOCHS}")

    # Training loop with tqdm
    for (batch, (inp, targ)) in enumerate(tqdm(train_dataset.take(train_steps_per_epoch), desc="Training", leave=False)):
        train_batch_loss = train_step(inp, targ, enc_hidden)
        train_loss += train_batch_loss

    # Validation loop with tqdm
    for (batch, (val_inp, val_tar)) in enumerate(tqdm(val_dataset.take(val_steps_per_epoch), desc="Validation", leave=False)):
        val_batch_loss = train_step(val_inp, val_tar, enc_hidden)
        val_loss += val_batch_loss

    # Saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 1 == 0:
        checkpoint.save(file_prefix=checkpoint_prefix)

    # Print train and validation loss
    print(f"Epoch {epoch + 1} Train Loss: {train_loss / train_steps_per_epoch:.4f}")
    print(f"Epoch {epoch + 1} Validation Loss: {val_loss / val_steps_per_epoch:.4f}")


Starting training from scratch.

Epoch 1/15


KeyboardInterrupt: 

# Load trained model

In [ ]:
checkpoint.restore("train_checkpoints/ckpt-10")

In [ ]:
# into base model
encoder = checkpoint.encoder
decoder = checkpoint.decoder

# Evaluation

Check bleu score

In [ ]:
def predict(sentence):
    inputs = tf.convert_to_tensor(sentence)
    result = ''
    inputs = tf.expand_dims(inputs, axis=0)
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, state = encoder(inputs, hidden)
    hidden_state = state
    dec_input = tf.expand_dims([target_lang_tokenize.word_index['start_']], 0)
    for t in range(max_length_target):
        predictions, hidden_state = decoder(dec_input,
                                                             hidden_state,
                                                             enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_lang_tokenize.index_word[predicted_id] + ' '
        if target_lang_tokenize.index_word[predicted_id] == '_end' or len(result) > max_length_target:
            return result

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
    return result

In [ ]:
def create_reference(lang, tensor):
    all_sentence_list = []

    for word_list in tensor:
      sentence_list = []

      for t in word_list:
          if not t == 0:
              # Index number assigned to each word
              sentence_list.append(lang.index_word[t])
      all_sentence_list.append(sentence_list)
    return all_sentence_list

In [ ]:
# create reference
reference = create_reference(target_lang_tokenize, Y_test.tolist()[:300])

In [ ]:
from tqdm import tqdm
# create hypothesis
hypothesis = []
for i in tqdm(X_test[:300]):
  hypothesis.append(predict(i))

In [ ]:
for ref, hyp in zip(reference, hypothesis):
  ref = " ".join(ref[1:-1])
  ss={
      "ref":ref,
      "hyp":hyp
  }
  # print(ref)
  print(ss)

In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

score = 0
smoothie = SmoothingFunction().method2
for i in range(len(reference)):
    score += sentence_bleu([reference[i][1:-1]], hypothesis[i][:-5].strip().split(), smoothing_function=smoothie)

score /= len(reference)
print("The bleu score is: "+str(score))

# Translation example

In [ ]:
def preprocess_sentence(en_text):
    # normalize Japanese
    ja_text = japanese_unicode_to_ascii(en_text)
    ja_text = replace_special_character_to_space(ja_text)
    ja_text = moji(ja_text)

    # add StTART and END sentence
    ja_text = "start_ " + ja_text + " _end"
    return ja_text

def evaluate(sentence):

    attention_plot = np.zeros((max_length_target, max_length_input))
    sentence = preprocess_sentence(sentence).strip()  # Remove extra spaces at start/end
    inputs = [input_lang_tokenize.word_index[i] for i in sentence.split(' ') if i in input_lang_tokenize.word_index]

    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_input,
                                                           padding='post')

    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, state = encoder(inputs, hidden)
    hidden_state = state
    dec_input = tf.expand_dims([target_lang_tokenize.word_index['start_']], 0)
    for t in range(max_length_target):
        predictions, hidden_state = decoder(dec_input, hidden_state, enc_out)
        predicted_id = tf.argmax(predictions[0]).numpy()

        result += target_lang_tokenize.index_word[predicted_id] + ' '
        if target_lang_tokenize.index_word[predicted_id] == '_end' or len(result) > max_length_target:
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence,

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

def result(sentence):
    result, sentence = evaluate(sentence)

    return result, sentence

In [ ]:
# result, sentence = result("he has a dog")
result, sentence = result("彼は犬を飼っています")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))

In [ ]:
# result, sentence = result("What is this!?")
result, sentence = result("これは何ですか!?")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))

In [ ]:
# result, sentence = result("Actually I did it")
result, sentence = result("実は、私がやりました。")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))

In [ ]:
# result, sentence = result("I love her")
result, sentence = result("彼女を愛しています。")
print('Input: %s' % (sentence))
print('Predicted translation: {}'.format(result))